In [ ]:
#https://stackoverflow.com/questions/47330244/generating-random-transition-probability-matrix-python
#https://docs.python.org/3/library/random.html
#ref: https://www.datacamp.com/tutorial/markov-chains-python-tutorial

In [ ]:
from Transition import Transition

In [ ]:
import datetime

Tra = Transition(3,3)

for _ in range(1):
    Tra.gen_state_prob(3, "uniform")
    M = Tra.get_transition_matrix()
    print(M)


print(M)

import numpy as np
from matplotlib import pyplot as plt
"""
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
im = plt.imshow(M, cmap="copper_r")
plt.colorbar(im)
plt.show()
"""
figure, axis = plt.subplots(3, 3)

for i in range(3):
    for j in range(3):
        Tra.gen_state_prob(3, "uniform")
        M = Tra.get_transition_matrix()
        plt.rcParams["figure.autolayout"] = True
        axis[i,j].imshow(M, cmap="copper_r")
        


#axis[0, 1].plot(X, Y2)
#axis[0, 1].set_title("Cosine Function")
#plt.savefig("plots/TM-heatmap-"+str(datetime.datetime.now())+".png", bbox_inches="tight")
plt.show()

## Plot State Transition Diagram

In [ ]:
'''
import numpy as np
from matplotlib import pyplot as plt

# Import the MarkovChain class from markovchain.py
from markovchain import MarkovChain
P = np.array(transition_matrix)
mc = MarkovChain(P, ['left', 'straight', 'right'])
mc.draw()
'''

## Simulate Map

In [ ]:
from MarkovProcess import MarkovChain
from Population import Population
from Simulate import Simulate

P = Population(20,0.5,0.5,30)
noOfTimeSteps = 40 - 2
#timeSteps = [10,50,100,150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950]
#timeSteps = [10,100,500,1000,1500,2000,2500,3000,3500,4000]
timeSteps = [10, 50]
timeVal = 50
for i in range(noOfTimeSteps):
    timeVal += 50
    timeSteps.append(timeVal)
print("timeSteps:", timeSteps)

S = Simulate(timeSteps)

S.addCars = True        #add new cars at every time step
S.addActCars = False

S.runSimulation(P)
print("lCars: ", S.getLCars())
print("timeSteps: ", S.getLCars())
#S.printMap(True)
S.plotData()
Features, Labels = S.getLearningData()
#[print(Features[i]) for i in range(5)] 
#print()
#[print(Labels[i]) for i in range(5)] 

S.generateTestingData(1000)
tFeatures, tLabels = S.getTestingData()
#print(tLabels)
#print(tFeatures)

#override features
#S.generateTestingData(1000)
#Features, Labels = S.getTestingData()

del S

In [ ]:
#https://vknight.org/unpeudemath/code/2015/08/01/simulating_continuous_markov_chains.html

## Plot MarkovChain

In [ ]:
'''
import pygraphviz as pgv
from IPython.display import Image

def draw(dot):
    return Image(pgv.AGraph(dot).draw(format='png', prog='dot'))

g1 = """digraph top {
   a -> b -> c;
}"""
draw(g1)
'''

## Decision Tree Classifier

In [ ]:
from sklearn import tree
X = Features
Y = Labels
pLabels = []
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

matched = 0
for i in range(len(tFeatures)):
    #print(clf.predict([tFeatures[i]]),"-->",tLabels[i])
    if clf.predict([tFeatures[i]]) == tLabels[i]:
        matched+=1
    pLabels.append(clf.predict([tFeatures[i]])[0])
        
#print(len(pLabels), len(tLabels))
print("accuracy: ", matched/len(tFeatures))

#clf.predict([[0.3,0.2,0.5,0.1,0.4,0.5,0.6,0.3,0.1,1]])
#clf.predict_proba([[0.3,0.2,0.5,0.1,0.4,0.5,0.6,0.3,0.1,1]])

In [ ]:
'''
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("traffic_tree") 
'''

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics

classifier = LogisticRegression()
classifier.fit(Features, Labels)
score = metrics.accuracy_score(Labels, classifier.predict(Features))
print("Accuracy: %f" % score)

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

## Neural Network MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
X = Features
y = Labels
MLPpLabels = []
    

print("data:",len(X))
print("labels:",len(y))

#scale learning data
dataX = X
scaler = MinMaxScaler()
print(scaler.fit(dataX))
dataX = scaler.transform(dataX)

[print(dataX[i]) for i in range(5)] 

clf = MLPClassifier(activation="relu",
                    solver='adam',#'lbfgs', 
                    alpha=0.00001,
                    hidden_layer_sizes=(12, ), 
                    random_state=1,
                    batch_size='auto',
                    early_stopping=False,
                    warm_start=True)

clf.fit(dataX, y)

#scale testing data
tDataX = tFeatures
scaler = MinMaxScaler()
print(scaler.fit(tDataX))
tDataX = scaler.transform(tDataX)

matched = 0
lbl = ""
for i in range(len(tDataX)):
    L = clf.predict([tDataX[i]])
    if L == tLabels[i]:
        matched+=1
    MLPpLabels.append(L[0])

print("accuracy: ", matched/len(tDataX))
print("score:", clf.score(dataX, y))
print("loss:", clf.loss_)

from matplotlib import pyplot as plt

plt.hist(tLabels, color='r')
#plt.legend()
plt.show()
plt.hist(MLPpLabels, color='b')
plt.show()
#print("tLabels", tLabels)
#print("MLPpLabels", MLPpLabels)

plt.plot(clf.loss_curve_, label="loss")
#plt.plot(clf.best_validation_score_, label="validation")

#plt.legend(True)
plt.show()



#clf.predict_proba([[2., 2.], [1., 2.]])

## Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import datetime

actual = tLabels
predicted = pLabels

#confusion_matrix = metrics.confusion_matrix(actual, predicted, labels=clf.classes_)#["left", "straight", "right"])
confusion_matrix = metrics.confusion_matrix(actual, MLPpLabels, labels=clf.classes_)#["left", "straight", "right"])
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = clf.classes_)#[False, True])

cm_display.plot(cmap = 'Blues')
#plt.savefig("plots/confusion-matrix-"+str(datetime.datetime.now())+".png", bbox_inches="tight")
plt.show() 

In [ ]:
#LATEST 01/19